In [1]:
import os 
%pwd
os.chdir('../')

In [2]:
%pwd

'e:\\ML_Projects_iNeuron\\End_end_Eye_Disease_detection_using_Mlops_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    train_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from cnnClassifier_Eye.constants import *
from cnnClassifier_Eye.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            train_data_path=Path(training.train_data_path),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.train_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.train_data_path,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2025-05-09 18:49:35,185: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-09 18:49:35,190: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-09 18:49:35,194: INFO: common: created directory at: artifacts]
[2025-05-09 18:49:35,198: INFO: common: created directory at: artifacts\training]
Found 98 images belonging to 2 classes.
Found 393 images belonging to 2 classes.
Epoch 1/10
24/24 [==============================] - 89s 4s/step - loss: 29.7789 - accuracy: 0.4456 - val_loss: 1.3706 - val_accuracy: 0.6146
Epoch 2/10
24/24 [==============================] - 96s 4s/step - loss: 20.9128 - accuracy: 0.4960 - val_loss: 6.3839 - val_accuracy: 0.5521
Epoch 3/10
24/24 [==============================] - 85s 4s/step - loss: 6.3751 - accuracy: 0.5570 - val_loss: 1.5499 - val_accuracy: 0.7604
Epoch 4/10
24/24 [==============================] - 104s 4s/step - loss: 11.1802 - accuracy: 0.5252 - val_loss: 5.6278 - val_accuracy: 0.6042
Epoch 5/10
24/24 [=======